# 10　数据存储：文件和HDF5

## HDF5

HDF5（Hierarchical Data Format 5，层次化数据存储格式）是一种自由开源的二进制文件类型规范。
HDF5已经成为科学计算中的默认二进制数据库。
科学家往往并不关心长度可变的字符串，而是处理高度结构化的数据。
区分数据的标准是数据的数量。

术语：exascale（百亿亿次级），1 EB（exabyte）=10^18^

HDF5不仅适用于大数据，也可用于微小数据集。

HDF支持C、Fortran、Java、C ++中的接口（C++接口大多已废弃，改用C接口）。
C 接口是默认的且功能最全的API。
在MATLAB、Mathematica、Haskell及其他语言中也有用来使用HDF4接口的第三方包。
**Python有两个包可使用HDF5：h5py和PyTables。**

## 10.1 Python中的文件

**需要与硬盘上的文件进行交互的情况：**
1.协作者通过电子邮件发送了原始数据。此时需要下载附件来查看结果。
2.需要快速通过电子邮件向协作者发送一些数据。
3.使用外部代码接受输入或数据文件。可能需要运行该程序数千次，因此用Python将内存中的数据自动生成为输入文件。
4.使用外部程序生成一个或多个结果文件，现在要读取这些文件来进行进一步的分析。
5.为了调试或验证而保留计算的中间结果。

读取和写入文件是一种交互。
交互中的发送者和接收者可以是人或其他程序，也可以两者都是。
文件作为这些交互的通用对象。
文件被进一步专门化为格式，指出文件的内部结构。
【特殊】.txt扩展名是一个例外，它不是一种格式。.txt一般用于表示文件没有特定的内部结构，但包含纯文本内容。

#### open()函数

以含有文件路径的字符串作为参数，生成一个句柄。

<code>f = open('data.txt')</code>执行的操作:
1．确保data.txt存在。
2．为该文件创建新的句柄。
3．将游标位置（pos）设为文件的起始处即pos = 0。

调用open()既不会读取文件，也不会写入或关闭文件。所有这些操作必须用其他方法操作文件句柄来单独完成。

#### 文件操作方法

**重要文件句柄方法**

| 方法            | 描述                                 |
|---------------|------------------------------------|
| f.read(n=-1)  | 从文件中读取n字节。如果不提供n或提供的值为-1，那么读取整个文件  |
| f.readline()  | 读取文件当前位置后的一整行内容，返回一个字符串，其中包括末尾的换行符 |
| f.readlines() | 读取文件中剩余的所有行，返回一个字符串列表，每个字符串都有换行符   |
| f.seek(pos)   | 将文件游标移到指定位置                        |
| f.tell()      | 返回游标在文件中位置                         |
| f.write(s)    | 在当前游标位置插入字符串                       |
| f.flush()     | 执行所有等待中的写入操作，确保其中的内容都写到磁盘上         |
| f.close()     | 关闭文件，之后就无法读取或写入内容                  |

从文件读取的行总是字符串。
必须将用字符串表示的矩阵中的值转换为整数。
Python仅假设数据类型是一个包含字符串的文件。用户必须告诉Python如何解释文件的内容。

在上面的代码片段的末尾需要手动关闭文件。
即使已经到达文件的末尾，Python也不会假定文件已经读取完毕。

#### 文件打开模式

给open()添加一个参数：
<code>f = open('data.txt', 'w')</code>

**有用的文件模式**

| 模式  | 含义                                              |
|-----|-------------------------------------------------|
| 'r' | 只读，无法写入。起始位置pos=0                               |
| 'w' | 写入。如果文件不存在则创建文件；如果文件存在，那么清空当前内容（注意！）。起始位置pos=0  |
| 'a' | 添加。打开文件写入内容，但不会删除当前内容。如果文件不存在则创建文件。起始位置pos是文件末尾 |
| '+' | 更新。打开文件供读取和写入；可与其他标志组合；不会删除当前内容。起始位置pos=0       |

#### 上下文管理器

**上下文管理：** 在有些情况下，无论一段代码最终执行成功还是失败，或是抛出异常，都必须在代码块的末尾运行一些特殊的安全代码，以此来防止数据丢失、损坏，或读取文件时最后停留在文件系统上错误的位置。
上下文管理器用于在进入代码块后，运行其中第一条语句之前执行一些防御性的操作，并在退出代码块时执行一些清理操作。
代码块在执行完最后一个语句或抛出未捕获异常之后就退出。
**文件句柄是Python中最常见的上下文管理器。**

with语句：用来进入和退出上下文。

格式：

```
with <context-manager> as <var>:  # 将上下文管理器对象赋值给局部变量
    <with-block>  # 管理器打开时执行的代码
```

用with语句使用文件，因为这样不必显式地调用f.close()，让代码更安全健壮。

## 10.2 关于计算机架构的一些说明

计算机的主要组件：
1.中央处理单元（CPU）
2.随机存取存储器（RAM）
3.存储驱动器（由磁盘驱动器（HDD）或固态驱动器（SSD）替代）

计算机的最简模型：
$$ CPU \leftrightarrow RAM \leftrightarrow Storage $$

CPU是一个哑的计算器；
RAM用来“记住”CPU刚刚做的事情（类似于短期记忆）
存储设备用来让计算机保存数据，即使计算机关闭，数据依然存在（类似于长期记忆）

CPU缓存：CPU上的小型RAM，含有RAM中的一些数据的副本，但比RAM更接近处理器。
对于经常访问的数据，缓存可以大大减少执行时间。
缓存用层次（level）加层次序号命名，例如L1、L2等。通常层次序号数字越高，缓存容量越小，访问速度越快。

图形处理单元（GPU）：通常称为显卡。位于主CPU外部的处理器。
GPU非常适合做浮点运算，而CPU适合做整数运算（同时也能很好地处理浮点数据）。
如果有一个主要由浮点数组成的应用，那么用GPU可能会加快执行速度。

## 10.3 HDF5中的重要思想

与用纯文本ASCII文件相比，结构化的数值数据更适合用二进制格式存储。
原生格式在空间方面几乎总是优于对应的字符串。
二进制格式I/O更快，因为在做数学计算时，字符串形式还要先转换为原生形式。

文件中不仅要有二进制数据，还要包括其他东西。
HDF5提供了常见的数据库功能，如存储多个数据集、让用户定义元数据、I/O 优化、查询功能等。
每个数据集都位于单个命名空间中，HDF5中的数据集可以存在于嵌套的树结构中。

PyTables提供了下面这些基本数据集类，可作为各种HDF5结构的入口点：
1.Array：文件系统的文件。
2.CArray：分块的数组。
3.EArray：可扩展数组。
4.VLArray：变长数组。
5.Table：结构数组。

所有这些类都必须由PyTables中所谓的原子类型（atomic type）组成，包括：
1.bool：True或False类型，有8位（bit）。
2.int：有符号整数类型，有8、16,、32（默认）、64位。
3.uint：无符号整数类型，有8、16,、32（默认）、64位。
4.float：浮点类型，有16、32、64（默认）位。
5.complex：复数浮点类型，有64和128（默认）位。
6.string：固定长度的原始字符串类型，8位乘以字符串长度。

层次结构可能还包括其他元素：
1.组（groups）：文件系统的目录，可能含有其他group和数据集。
2.链接（links）：相当于文件系统中的软链接。
3.隐藏节点（hidden nodes）：类似隐藏文件。

## 10.4 文件处理

可以用PyTables接口打开HDF5文件。
为了获取PyTables，首先要导入tables。

**PyTables打开文件的模式：**

| 属性 | 说明                             |
|----|--------------------------------|
| r  | 只读，无法修改数据。                     |
| w  | 写入，创建新文件，如果已有同名文件，则该文件会被删除。    |
| a  | 添加，打开已有的文件以供读写，如果文件不存在则创建该问题就。 |
| r+ | 与添加相似，但文件必须已经存在。               |

在HDF5中，所有节点都来自根节点“/”或f.root。
层次结构中的子节点是作为上一级节点的属性来访问。
新节点必须在文件句柄上创建，不能在节点上创建。
要创建一个新的组，必须对文件调用create_group()方法，然后在相同的位置访问该组。

文件句柄上分别以create_array()和create_table()方法来创建数组（array）和表（table）。
数组大小固定，因此在创建时就必须提供数据。
表类似NumPy的结构化数组，是一种集合数据类型，长度是可变的。

数组和表会保留创建时所具有的特性或数据结构。
使用Python 列表创建的数据集，则读取数据时将返回一个Python列表。
使用NumPy结构化数组来生成数据，那么将返回NumPy结构化数组。
通过切片能够读取数据集中的数据。
在进行切片操作时，只会从磁盘读取所需的元素，不会触及数据集中未包含在切片中的部分。

由于arthur_count数组来自一个Python列表，因此只能使用Python列表形式的切片。
但如果数据集最初来自NumPy数组，那么就可以用NumPy风格访问，包括切片、花式索引、掩模。

切片方式是内存映射，按需提取，不访问整个数组。


In [ ]:
import tables as tb
import numpy as np


f = tb.open_file('/path/to/file', 'a')  # 文件打开为句柄
f.create_group('/', 'a_group', "My Group")  # 创建组
# f.root.a_group  # 访问根节点上的组

# 创建数组和表，以及如何向其中添加Python列表和NumPy数组
f.create_array('/a_group', 'arthur_count', [1, 2, 5, 3])  # 创建数组数据集
dt = np.dtype([('id', int), ('name', 'S10')])  # 定义数据类型
knights = np.array([(42, 'Lancelot'), (12, 'Bedivere')], dtype=dt)  # 用dt格式创建数据集
f.create_table('/', 'knights', dt)  # 用knight的数据创建一个表格数据集
f.root.knights.append(knights)

# 文件中组和数据集的层次结构
"""
/
|-- a_group/
| |-- arthur_count
|
|-- knights
"""
# 输出结果
print(f.root.a_group.arthur_count[:])
print(type(f.root.a_group.arthur_count[:]))
print(type(f.root.a_group.arthur_count))

# NumPy风格的接口
print(f.root.knights[1])  # 获取第二行
print(f.root.knights[:1])  # 对第一行切片
mask = (f.root.knights.cols.id[:] < 28)
print(f.root.knights[mask])  # 根据磁盘上的id列创建掩模，将其应用到表中
print(f.root.knights[([1, 0],)])  # 按照先2后1的顺序对前两行进行花式索引

## 10.5 层次布局

对于大量、复杂的数据，根据其特征，对其进行层次结构的划分，以降低访问的复杂度。
可进行分组、分表等。
这样做的优点：
1.需要搜索的行数更少。
2.从磁盘拉取的行也更少。
3.描述信息中的列较少，因而减少了行的大小。

调整层次结构的目的：
1.降低计算机到硬盘的访问次数
2.减少从硬盘传输的数据量
3.在计算机等待数据期间保持计算机繁忙状态

在实际中，层次的深度和广度有限制。
每个新的组或数据集都有管理开销。
在HDF5中，每个数据集的元数据需要占用64 KB的空间，当管理开销大小接近存储的基础数据大小时，就遇到了深度和广度限制。

**推荐使用三层结构的数据集**

## 10.6 分块

分块：将数据集拆分成较小部分，发生在单个数据集中，并且是自动实现的。
HDF5用分块方式存储数据，**此特性与Numpy无对应。**
分块需要在文件中每个块的实际位置存储额外的元数据。

**分块让HDF5具有处理大量数据的两个关键功能：**
1．能够有效存储稀疏数据。
2．数据集可以在所有维度上无限延伸。

【例】假设想要存储109×109项目矩阵。如果这个矩阵主要由零组成，则可以使用HDF5仅存储非零元素。

PyTables通常仅允许在行这个维度扩展。
HDF5本身没有行列的限制，使用HDF5的C接口（或h5py可以实现在所有维度上无限扩展。

连续数据集中，所有字节都在单个数组中，彼此相邻。
分块数据集可以将字节数组分割成许多较小的数组，每个数组存储在一个单独的位置。
在HDF5中，所有读写操作都发生在各个分块中。

块形状和大小是数据集本身的属性，与实际数据无关。
**性质的重要影响：**
1.边缘块可以扩展到数据集之外。
2.在未分配空间中设置默认填充值，允许稀疏数据集仅存储其所拥有的数据。
3.块的读取和写入可能并行发生。
4.小块适用于每次只访问一小部分数据。
5.大块适合于一次访问大量数据。

HDF5 中的一些数据存储操作方式也是因分块而存在。
允许用户在所有分块数据集上设置块大小和形状。
PyTables中，创建时可以使用chunkshape关键字参数，这是一个元组，表示每个维度的块的形状和大小。可以与create_carray()方法关联：
<code>f.create_carray('/', 'omnomnom', data, chunkshape=(42,42))</code>
没有明确设置块形状，PyTables将根据数据的属性和一些信息来猜测最佳分块方式。

#### 分块的工作方式

假设内存中有一个3×3矩阵：
存储为连续数据集，需要一个具有9个元素的平面数组。
可以自由选择小于或等于3×3的任何形状的块。
所有块的总形状可以超过原始数据的边界，超出边界的块元素不会用到。

【例】取2×2作为块的形状，然后3×3矩阵将由4个能覆盖到矩阵的2×2块表示。

调整块形状能够微调应用程序性能。

## 10.7 核内和核外操作

实现数组操作时，根据期望数据在内存中的位置，算法可分为两大类：
核内（incore）操作：需要所有数据在存储器中的操作，可能受存储器限制，可以处理的数据量限制为计算机内存的大小。
核外（out-of-core）操作：在存储器外部存储的数据进行的操作，可能是CPU密集型的操作，

可以根据组件（RAM或CPU）限制机器的行为来区分“核内”和“核外”。
HDF5数组上的计算可以以核外方式执行。

### 10.7.1 核内

假设内存中有两个NumPy数组a和b，c的表达式与a和b的大小和内容无关。只要a和b是兼容的就可以计算c。

c的表达式创建三个临时数组，在使用后都会丢弃，但这些临时数组占用了存储器中的空间。因此计算c时需要内存中至少有a或b大小的六倍的空间。

使用内存较少的方式是逐元素地计算整个表达式。C、C ++或Fortran中经常使用这种方式的操作，但Python的速度要慢得多。（原因：1、Python中的循环很慢；2、Python使用动态类型系统）
需要加速：用其它编程语言。


In [4]:
import numpy as np


a = np.array([1, 2, 3, 4, 5])
b = np.array([9, 8, 7, 6, 5])

# numpy直接计算的方法
c = 42 * a + 28 * b + 6
# temp1 = 42 * a  # 第一个临时值
# temp2 = 28 * b  # 第二个临时值
# temp3 = temp1 + temp2  # 临时值相加
# c = temp3 + 6  # c值的计算
print(c)

# 内存开销更小的计算方法
c = np.empty(shape=a.shape)
for i in range(len(c)):
    c[i] = 42 * a[i] + 28 * b[i] + 6
print(c)



[300 314 328 342 356]
[300. 314. 328. 342. 356.]


### 10.7.2 核外

#### 核外计算策略

核外操作结合了逐元素计算和分块的概念。
用于处理完全在存储器内部的数据的。

核外算法会将大小为256的分块应用于逐元素算法。
使用核外策略时，在任何时候内存中不需要超过256个元素。当创建临时对象时，大小也限制为256个元素。每次只提供离散的块。
适合处理并行数据。

#### numexpr库

numexpr库提供了一种对常规NumPy数组执行分块、逐元素计算的方法。
PyTables在内部使用此库，并将其扩展到磁盘上的HDF5数组。
PyTables的tb.Expr类实现了处理表达式求值的核外接口。
c的表达式可以用Expr类和以位于存储器外部的HDF5数组表示的a 和b来计算。
与编写自定义分块算法相比，使用tb.Expr或numexpr更加清楚、简单、强大。

【总结】
1.当所有数据都在内存中时，核内操作非常快速高效。
2.如果数据需要从磁盘读取，或者因为数据太大而不能放在内存中，则核外算法是最好的。
3.对于HDF5，PyTables提供了Expr类来执行核外操作。


In [7]:
# 核外计算策略
import numpy as np


# 设置初始数据
a = np.array([1, 2, 3, 4, 5])
b = np.array([9, 8, 7, 6, 5])

# 核外方法计算
c = np.empty(shape=a.shape)
for i in range(0, len(a), 256):
    r0, r1 = a[i:i+256], b[i:i+256]  # 计算基础矩阵
    r2 = np.multiply(r0, 42)  # 第一个中间变量
    r3 = np.multiply(r1, 28)  # 第二个中间变量
    np.add(r2, r3, r2)
    np.add(r2, 6, r2)
    c[i:i+256] = r2
print(c)

[300. 314. 328. 342. 356.]


In [ ]:
# numexpr库的核外计算策略
import tables as tb


# 打开文件
shape = (10, 10000)
f = tb.open_file("/tmp/expression.h5", "w")
# 创建数组
a = f.create_carray(f.root, 'a', tb.Float32Atom(dflt=1.), shape)
b = f.create_carray(f.root, 'b', tb.Float32Atom(dflt=2.), shape)
c = f.create_carray(f.root, 'c', tb.Float32Atom(dflt=3.), shape)
# 计算，输出c
expr = tb.Expr("42*a + 28*b + 6")
expr.set_output(c)
expr.eval()
# close the file
f.close()

## 10.8 查询

查询：询问现有数据集中某一元素是否满足一些标准。

PyTables的Table类中的查询方法：

```python
tb.Table.where(cond) # 返回匹配项的迭代器
tb.Table.get_where_list(cond) # 返回索引列表
tb.Table.read_where(cond) # 返回结果列表
tb.Table.append_where(dest, cond) # 将匹配结果添加到另一个表中
```

查询调用中使用的条件会自动转为对表中每行进行的核外操作。

cond表达式必须返回布尔值。
如果表达式为True，那么该行将包含在输出中。
如果条件为False，则跳过该行。

条件在HDF5文件中的列名称和其他数据集的上下文中执行。
where()方法为所有匹配上的行返回一个迭代器，经常会见到将表中的列或特定值进行比较。

查询可能相当复杂。此时核外计算带来的优势就显得至关重要。这种情况下临时变量更少，需要从磁盘单独传输的数据也更少。

## 10.9 压缩

CPU饥饿问题：处理器在从磁盘读取数据时不会做任何事情。
解决方法之一：压缩
当数据集写入磁盘时通过压缩算法压缩，当从磁盘读取数据时是解压。
压缩在每个块上独立发生。

压缩算法本身会带来一些存储开销，然而，即使有压缩开销，压缩也可以大大减少文件大小和访问时间。对于非常规则的数据，甚至可以将数据大小减少到接近零。
压缩和解压缩显然需要更多的处理能力，而不是简单地将数组直接从磁盘复制到内存中。但因为传输到磁盘和从磁盘传输的总信息较少，所以解压缩数组所花费的时间远小于批量移动数组所需的时间。

**HDF5的压缩**
压缩是HDF5本身的一个特性，依赖zlib库。
通过插件架构实现。
允许使用多种不同的算法，包括用户定义的算法。
PyTables支持：zlib（默认）、lzo、bzip2、blosc。

压缩通过名为过滤器的机制启用。
过滤器位于内存中的数据和磁盘上的数据之间，对通过的所有数据执行转换。

**PyTables.Filter类**
允许设置压缩算法名称（以字符串）和压缩级别，级别是从0（无压缩）到9（最大压缩）的整数。
PyTables中，可以在任何组或数据集上或文件本身上设置过滤器。如果未指定，数据集或组将从层次结构中直接位于其上方的节点继承过滤器。

过滤器的应用方法：

```python
# 从 [0,9] 开始
filters = tb.Filters(complevel=5, complib='blosc')  # 创建filters对象
# 对整个文件设置过滤器
f = tb.open_file('/path/to/file', 'a', filters=filters)  # 在创建文件后设置过滤器
f.filters = filters  # 在打开文件后设置过滤器
# 过滤器也可在大多数其他节点上设置
f.create_table('/', 'table', desc, filters=filters)  # 在创建数据集后设置过滤器
f.root.group._v_filters = filters  # 对已有数据集设置过滤器
```

在选择压缩参数时，中级（5）压缩通常就足够了。没有必要一直到最大压缩（9），这样会消耗更多的处理器时间而空间的提升却很小。

保证完全的可移植性使用zlib。
只用PyTables则使用blosc。

## 10.10 HDF5工具

HDF5以各种编程语言访问数据，但每次检查HDF5文件时，编写程序来查看HDF5文件的内容可能是个非常大的工作。因此要使用工具。

用于查找HDF5文件的主要命令行工具：
1.h5ls。针对HDF5文件的类ls工具，来自HDF5。可以查看相关文件、层次结构及其数据集的元数据。
2.h5dump。将HDF5表的内容全部或部分打印到屏幕，来自HDF5。
3.ptdump。PyTables版的h5dump，来自PyTables。
三个程序都使用HDF5文件作为参数。

```
     h5ls 2srcs3rxts.h5
AgentDeaths                  Dataset {7/Inf}
Agents                       Dataset {7/Inf}
Compositions                 Dataset {200/Inf}
InputFiles                   Dataset {1/Inf}
ResCreators                  Dataset {23/Inf}
Resources                    Dataset {85/Inf}
SimulationTimeInfo           Dataset {1/Inf}
Transactions                 Dataset {17/Inf}
```

第一列包含层次结构节点名称。
第二列列出了节点类型，由于没有组，所以这里的每个节点都是一个数据集。
第三列包含数据集大小。
斜杠之前的数字是当前大小，斜杠后面的数字是最大值。
这里的Inf 表示所有数据集都是可扩展的，因此大小无限。

交互式探索HDF5文件的图形工具:
1.hdfview是基于Java的查看器和HDF组编辑器，能更好地处理在许多维度上无限延伸的数据集。
2.ViTables是独立的，是基于PyTables和Qt的HDF5查看器，能更好地显示大量的数据，并且为PyTables中的查询功能提供了一个图形界面。


